In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


Task 1:

In the accounts table, there is a column holding the website for each company. The last three digits specify what type of web address they are using. A list of extensions (and pricing) is provided here. Pull these extensions and provide how many of each website type exist in the accounts table.

In [6]:
# sqlite version

pd.read_sql_query(sql='''
SELECT SUBSTR(website, LENGTH(website)-2, 3) extension, COUNT(*) count
FROM accounts
GROUP BY 1;
''', con=conn)

,extension,count
0,com,349
1,net,1
2,org,1


Solution:

SELECT RIGHT(website, 3) AS domain, COUNT(*) num_companies
FROM accounts
GROUP BY 1
ORDER BY 2 DESC;

Task 2:

There is much debate about how much the name (or even the first letter of a company name) matters. Use the accounts table to pull the first letter of each company name to see the distribution of company names that begin with each letter (or number).

In [20]:
pd.read_sql_query(sql='''
SELECT SUBSTR(UPPER(name),1,1), COUNT (*)
FROM accounts
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,"SUBSTR(UPPER(name),1,1)",COUNT (*)
0,C,37
1,A,37
2,P,27
3,M,22
4,T,17
5,S,17
6,D,17
7,L,16
8,E,16
9,B,16


Solution:

SELECT LEFT(UPPER(name), 1) AS first_letter, COUNT(*) num_companies
FROM accounts
GROUP BY 1
ORDER BY 2 DESC;

Task 3:

Use the accounts table and a CASE statement to create two groups: one group of company names that start with a number and a second group of those company names that start with a letter. What proportion of company names start with a letter?

In [16]:
pd.read_sql_query(sql='''
SELECT CASE WHEN SUBSTR(name,1,1) NOT IN ('1','2','3','4','5','6','7','8','9','0') THEN 'LETTER' ELSE 'NUMBER' END start_with,
COUNT (*) count, name 
FROM accounts
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,start_with,count,name
0,LETTER,350,Walmart
1,NUMBER,1,3M


In [21]:
pd.read_sql_query(sql='''
SELECT SUM(num) nums, SUM(letter) letters
FROM (SELECT name, CASE WHEN SUBSTR(UPPER(name), 1,1) IN ('0','1','2','3','4','5','6','7','8','9') 
                       THEN 1 ELSE 0 END AS num, 
         CASE WHEN SUBSTR(UPPER(name), 1,1) IN ('0','1','2','3','4','5','6','7','8','9') 
                       THEN 0 ELSE 1 END AS letter
      FROM accounts) t1;
''', con=conn)

,nums,letters
0,1,350


Solution:

SELECT SUM(num) nums, SUM(letter) letters
FROM (SELECT name, CASE WHEN LEFT(UPPER(name), 1) IN ('0','1','2','3','4','5','6','7','8','9') 
                       THEN 1 ELSE 0 END AS num, 
         CASE WHEN LEFT(UPPER(name), 1) IN ('0','1','2','3','4','5','6','7','8','9') 
                       THEN 0 ELSE 1 END AS letter
      FROM accounts) t1;

Task 4:

Consider vowels as a, e, i, o, and u. What proportion of company names start with a vowel, and what percent start with anything else?

In [24]:
pd.read_sql_query(sql='''
SELECT CASE WHEN SUBSTR(name,1,1) IN ('a','e','i','o','u','A','E','I','O','U') THEN 'VOWEL' ELSE 'ELSE' END start_with,
COUNT (*) count, name 
FROM accounts
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,start_with,count,name
0,ELSE,271,Walmart
1,VOWEL,80,Exxon Mobil


In [25]:
pd.read_sql_query(sql='''
SELECT CASE WHEN SUBSTR(LOWER(name),1,1) IN ('a','e','i','o','u') THEN 'VOWEL' ELSE 'ELSE' END start_with,
COUNT (*) count, name 
FROM accounts
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,start_with,count,name
0,ELSE,271,Walmart
1,VOWEL,80,Exxon Mobil


In [26]:
pd.read_sql_query(sql='''
SELECT SUM(vowels) vowels, SUM(other) other
FROM (SELECT name, CASE WHEN SUBSTR(UPPER(name), 1,1) IN ('A','E','I','O','U') 
                        THEN 1 ELSE 0 END AS vowels, 
          CASE WHEN SUBSTR(UPPER(name), 1,1) IN ('A','E','I','O','U') 
                       THEN 0 ELSE 1 END AS other
         FROM accounts) t1;
''', con=conn)

,vowels,other
0,80,271


Solution:

SELECT SUM(vowels) vowels, SUM(other) other
FROM (SELECT name, CASE WHEN LEFT(UPPER(name), 1) IN ('A','E','I','O','U') 
                        THEN 1 ELSE 0 END AS vowels, 
          CASE WHEN LEFT(UPPER(name), 1) IN ('A','E','I','O','U') 
                       THEN 0 ELSE 1 END AS other
         FROM accounts) t1;